In [18]:
import os
os.chdir("/Users/jacksonwalters/Documents/GitHub/enefit-kaggle/predict-energy-behavior-of-prosumers/")
!ls

client.csv                            gas_prices.csv
county_id_to_name_map.json            historical_weather.csv
electricity_prices.csv                public_timeseries_testing_util.py
enefit                                train.csv
example_test_files                    weather_station_to_county_mapping.csv
forecast_weather.csv


In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from load_data import merged_df

In [20]:
df = merged_df()

In [22]:
#drop the target column and data_block_id as they are not present in test data
X = df.drop(['target'], axis=1)

In [23]:
#set the target variable
y = df['target']

In [9]:
#verify there are no NaN's
np.any(np.isnan(y))

False

In [24]:
# load the modeling modules from sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.metrics import mean_absolute_error

In [25]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [26]:
#create a multivariable linear regression
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
regr.feature_names = list(X_train.columns.values)

In [27]:
#predict on the test data
lin_pred = regr.predict(X_test)

In [28]:
mean_absolute_error(lin_pred,y_test)

286.9963262675203

In [29]:
#train the random forest regressor
rf = RandomForestRegressor(n_estimators=1)
rf.fit(X_train, y_train)
rf.feature_names = list(X_train.columns.values)

In [30]:
#predict the target values using the random forest regressor
rf_pred = rf.predict(X_test)

In [31]:
#mean absolute error for random regressor
mean_absolute_error(y_test,rf_pred)

0.20607559902690709

In [33]:
#perform k-fold cross-validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #train the model
    rf_regr = RandomForestRegressor(n_estimators=1)
    rf_regr.fit(X_train, y_train)
    #predict on the test data
    rf_pred = rf_regr.predict(X_test)
    #evaluate the model
    print(mean_absolute_error(rf_pred,y_test))

68.74932818398159
44.79244121989514
43.59930557934766
50.11175003921031
51.49235091762749


In [40]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
# Perform time series split cross-validation
cross_val_scores = cross_val_score(rf, X, y, cv=tscv, scoring='neg_mean_absolute_error')

In [41]:
cross_val_scores

array([-50.24322373, -61.66098868, -36.89642564, -62.35650541,
       -55.67914648])

In [43]:
import pickle
# save the linear model to disk
linear_filename = '../models/linear_model.sav'
pickle.dump(regr, open(linear_filename, 'wb'))
random_forest_filename = '../models/random_forest_model.sav'
pickle.dump(rf, open(random_forest_filename, 'wb'))